In [8]:
import pandas as pd 
import numpy as np
import plotly_express as px
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import classification_report

In [7]:
pd.set_option("display.Max_columns",None)
df = pd.read_csv("Base para validar IBILLETE.csv")
df.head(2)

,idconsulta,bd,edad,estado,age_range,score,nodo,Risk_Code,idcreidit,Solución Original,Observacion,DEPENDIENTES,Codigo Razón 1,delegacionMunicipio,tipoDomicilio,identificadorSociedadInformacionCrediticia,indicadorTipoResponsabilidad,tipoCuenta,tipoContrato,numeroPagos,frecuenciaPagos,cuentasRevolventesAbiertas,cuentasCerradas,cuentasNegativasActuales,cuentasClavesHistoriaNegativa,numeroSolicitudesUltimos6Meses,numeroTotalCuentasDespachoCobranza
0,2550443656,33648,30,EM,2,551,10,Seguir con el proceso Fast Track Monto Máximo ...,990093,Solucion_18,atraso credito,0,15,TULTITLAN,H,1,I,I,PL,1,M,6,89,1,1,54,0
1,2538689645,25104,54,CHI,5,551,10,Seguir con el proceso Fast Track Monto Máximo ...,979020,Solucion_18,sin dias de atraso,0,7,JUAREZ,0,1,I,I,PL,1,M,14,36,17,1,24,0


In [15]:
#Cambiamos las observaciones por datos númericos 
#df["Observacion"] = np.where(df["Observacion"]=="atraso credito",1,0)
df.head(2)

In [18]:
df.shape


(576, 27)

In [19]:
df_train,df_test = train_test_split(df, test_size=0.25,random_state=0)
print('Size of train dataframe: ', df_train.shape[0])
print('Size of test dataframe: ', df_test.shape[0])

Size of train dataframe:  432
Size of test dataframe:  144


In [20]:
# Create a flag for label masking
df_train['Random_Mask'] = True
df_train.loc[df_train.sample(frac=0.05, random_state=0).index, 'Random_Mask'] = False

In [21]:
# Create a new target colum with labels. The 1's and 0's are original labels and -1 represents unlabeled (masked) data
df_train['Dependents_Target']=df_train.apply(lambda x: x['Observacion'] if x['Random_Mask']==False else -1, axis=1)

In [23]:
# Show target value distribution
print('Target Value Distribution:')
print(df_train['Dependents_Target'].value_counts())

Target Value Distribution:
-1    410
 0     17
 1      5
Name: Dependents_Target, dtype: int64


In [30]:
df_train[["Observacion","Random_Mask","Dependents_Target"]]

,Observacion,Random_Mask,Dependents_Target
140,0,True,-1
521,0,True,-1
404,0,True,-1
113,1,True,-1
378,0,True,-1
...,...,...,...
277,1,True,-1
9,1,True,-1
359,0,True,-1
192,0,True,-1


In [67]:
#df_train.iloc[:,2:7]
#df_train.dtypes
df_train[["edad","age_range","score","nodo", "DEPENDIENTES","Codigo Razón 1","identificadorSociedadInformacionCrediticia","indicadorTipoResponsabilidad","numeroPagos","cuentasRevolventesAbiertas","cuentasCerradas","cuentasNegativasActuales","cuentasClavesHistoriaNegativa","numeroSolicitudesUltimos6Meses","numeroTotalCuentasDespachoCobranza"]].dtypes	

edad                                           int64
age_range                                      int64
score                                          int64
nodo                                           int64
DEPENDIENTES                                   int64
Codigo Razón 1                                 int64
identificadorSociedadInformacionCrediticia     int64
indicadorTipoResponsabilidad                  object
numeroPagos                                    int64
cuentasRevolventesAbiertas                     int64
cuentasCerradas                                int64
cuentasNegativasActuales                       int64
cuentasClavesHistoriaNegativa                  int64
numeroSolicitudesUltimos6Meses                 int64
numeroTotalCuentasDespachoCobranza             int64
dtype: object

In [59]:
df_train["estado"] = (df_train["estado"]).astype(str)

In [74]:
########## Step 1 - Data Prep ########## 
# Select only records with known labels
df_train_labeled=df_train[df_train['Dependents_Target']!=-1]

# Select data for modeling 
#,"estado",,"delegacionMunicipio","tipoContrato","frecuenciaPagos","tipoCuenta","tipoDomicilio",,"indicadorTipoResponsabilidad"
X_baseline = df_train_labeled[["edad","age_range","score","nodo", "DEPENDIENTES","Codigo Razón 1","identificadorSociedadInformacionCrediticia","numeroPagos","cuentasRevolventesAbiertas","cuentasCerradas","cuentasNegativasActuales","cuentasClavesHistoriaNegativa","numeroSolicitudesUltimos6Meses","numeroTotalCuentasDespachoCobranza"]]
y_baseline = df_train_labeled['Dependents_Target'].values

# Put test data into an array
X_test=df_test[["edad","age_range","score","nodo", "DEPENDIENTES","Codigo Razón 1",	"identificadorSociedadInformacionCrediticia","numeroPagos","cuentasRevolventesAbiertas","cuentasCerradas","cuentasNegativasActuales","cuentasClavesHistoriaNegativa","numeroSolicitudesUltimos6Meses","numeroTotalCuentasDespachoCobranza"]]
y_test=df_test['Observacion'].values


########## Step 2 - Model Fitting ########## 
# Specify SVC model parameters
model = SVC(kernel='rbf', #rbf
            probability=True, 
            C=1, # default = 1.0
            gamma='scale', # default = 'scale'
            random_state=0
           )

# Fit the model
clf = model.fit(X_baseline, y_baseline)


########## Step 3 - Model Evaluation ########## 
# Use score method to get accuracy of the model
print('---------- SVC Baseline Model - Evaluation on Test Data ----------')
accuracy_score_B = model.score(X_test, y_test)
print('Accuracy Score: ', accuracy_score_B)
# Look at classification report to evaluate the model
print(classification_report(y_test, model.predict(X_test)))

---------- SVC Baseline Model - Evaluation on Test Data ----------
Accuracy Score:  0.7638888888888888
              precision    recall  f1-score   support

           0       0.76      1.00      0.87       110
           1       0.00      0.00      0.00        34

    accuracy                           0.76       144
   macro avg       0.38      0.50      0.43       144
weighted avg       0.58      0.76      0.66       144



c:\Users\RobertoRosasGuerrero\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\RobertoRosasGuerrero\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\RobertoRosasGuerrero\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [75]:
########## Step 1 - Data Prep ########## 
# Select data for modeling - we are including masked (-1) labels this time


X_train = df_train[["edad","age_range","score","nodo", "DEPENDIENTES","Codigo Razón 1","identificadorSociedadInformacionCrediticia","numeroPagos","cuentasRevolventesAbiertas","cuentasCerradas","cuentasNegativasActuales","cuentasClavesHistoriaNegativa","numeroSolicitudesUltimos6Meses","numeroTotalCuentasDespachoCobranza"]]
y_train = df_train['Dependents_Target'].values


########## Step 2 - Model Fitting ########## 
# Specify SVC model parameters
model_svc = SVC(kernel='rbf', 
                probability=True, # Need to enable to be able to use predict_proba
                C=1.0, # default = 1.0
                gamma='scale', # default = 'scale',
                random_state=0
               )

# Specify Self-Training model parameters
self_training_model = SelfTrainingClassifier(base_estimator=model_svc, # An estimator object implementing fit and predict_proba.
                                             threshold=0.7, # default=0.75, The decision threshold for use with criterion='threshold'. Should be in [0, 1).
                                             criterion='threshold', # {‘threshold’, ‘k_best’}, default=’threshold’, The selection criterion used to select which labels to add to the training set. If 'threshold', pseudo-labels with prediction probabilities above threshold are added to the dataset. If 'k_best', the k_best pseudo-labels with highest prediction probabilities are added to the dataset.
                                             #k_best=50, # default=10, The amount of samples to add in each iteration. Only used when criterion='k_best'.
                                             max_iter=100, # default=10, Maximum number of iterations allowed. Should be greater than or equal to 0. If it is None, the classifier will continue to predict labels until no new pseudo-labels are added, or all unlabeled samples have been labeled.
                                             verbose=True # default=False, Verbosity prints some information after each iteration
                                            )

# Fit the model
clf_ST = self_training_model.fit(X_train, y_train)


########## Step 3 - Model Evaluation ########## 
print('')
print('---------- Self Training Model - Summary ----------')
print('Base Estimator: ', clf_ST.base_estimator_)
print('Classes: ', clf_ST.classes_)
print('Transduction Labels: ', clf_ST.transduction_)
#print('Iteration When Sample Was Labeled: ', clf_ST.labeled_iter_)
print('Number of Features: ', clf_ST.n_features_in_)
print('Feature Names: ', clf_ST.feature_names_in_)
print('Number of Iterations: ', clf_ST.n_iter_)
print('Termination Condition: ', clf_ST.termination_condition_)
print('')

print('---------- Self Training Model - Evaluation on Test Data ----------')
accuracy_score_ST = clf_ST.score(X_test, y_test)
print('Accuracy Score: ', accuracy_score_ST)
# Look at classification report to evaluate the model
print(classification_report(y_test, clf_ST.predict(X_test)))


End of iteration 1, added 410 new labels.

---------- Self Training Model - Summary ----------
Base Estimator:  SVC(probability=True, random_state=0)
Classes:  [0 1]
Transduction Labels:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

c:\Users\RobertoRosasGuerrero\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\RobertoRosasGuerrero\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\RobertoRosasGuerrero\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

